In [41]:
'''
VGG11/13/16/19 in TensorFlow2.

Reference:
[1] Simonyan, Karen, and Andrew Zisserman. 
    "Very deep convolutional networks for large-scale image recognition." 
    arXiv preprint arXiv:1409.1556 (2014).
'''
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input
from tensorflow.keras.layers import Flatten, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
import numpy as np
import math
from sklearn.model_selection import train_test_split
config = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

from tensorflow.keras import layers, Model, Input

def build_vgg_functional(vgg_name, num_classes):
    inputs = Input(shape=(32, 32, 3))
    x = inputs
    for l in config[vgg_name]:
        if l == 'M':
            x = layers.MaxPooling2D(pool_size=2, strides=2)(x)
        else:
            x = layers.Conv2D(l, kernel_size=3, padding='same')(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)

    x = layers.AveragePooling2D(pool_size=1, strides=1)(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return Model(inputs, outputs)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    
    return lr

class trainer():
    def __init__(self, path, x_tr, y_tr, x_v, y_v, model_name,num_classes,subtract_pixel_mean = True,):
        assert len(y_tr.shape)==2
        assert len(y_v.shape)==2
        assert len(x_tr.shape)==4
        assert len(x_v.shape)==4
        
        self.path = path #"path_of_folder/" for saving and loading
        
        # if subtract pixel mean is enabled
        if subtract_pixel_mean:
            x_train_mean = np.mean(x_tr, axis=0)
            x_tr -= x_train_mean
            x_v -= x_train_mean          
        
        self.x_tr = x_tr
        self.y_tr = y_tr
        self.x_v = x_v
        self.y_v = y_v
        self.subtract_pixel_mean = subtract_pixel_mean
        self.model = build_vgg_functional(model_name, num_classes=num_classes)
        self.model.save(path)
    def train(self, batch_size = 32, epochs = 200, 
              data_augmentation = True, optimizer=None,loss='categorical_crossentropy', 
              save_best_only=False,save_weights_only=False):
              
        if optimizer is None:
              optimizer=Adam(learning_rate=lr_schedule(0))
              
        self.optimizer = optimizer
        self.model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=['acc'])
              
        # prepare callbacks for model saving and for learning rate adjustment.
        checkpoint = ModelCheckpoint(filepath=self.path,
                                     monitor='val_acc',
                                     verbose=1,
                                     save_best_only=save_best_only, save_weights_only=save_weights_only)

        lr_scheduler = LearningRateScheduler(lr_schedule)

        lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                       cooldown=0,
                                       patience=5,
                                       min_lr=0.5e-6)

        callbacks = [checkpoint, lr_reducer, lr_scheduler]

        # run training, with or without data augmentation.
        if not data_augmentation:
            print('Not using data augmentation.')
            self.model.fit(self.x_tr, self.y_tr,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_data=(self.x_v, self.y_v),
                      shuffle=True,
                      callbacks=callbacks)
        else:
            print('Using real-time data augmentation.')
            # this will do preprocessing and realtime data augmentation:
            datagen = ImageDataGenerator(
                # set input mean to 0 over the dataset
                featurewise_center=False,
                # set each sample mean to 0
                samplewise_center=False,
                # divide inputs by std of dataset
                featurewise_std_normalization=False,
                # divide each input by its std
                samplewise_std_normalization=False,
                # apply ZCA whitening
                zca_whitening=False,
                # randomly rotate images in the range (deg 0 to 180)
                rotation_range=0,
                # randomly shift images horizontally
                width_shift_range=0.1,
                # randomly shift images vertically
                height_shift_range=0.1,
                # randomly flip images
                horizontal_flip=True,
                # randomly flip images
                vertical_flip=False)

            # compute quantities required for featurewise normalization
            # (std, mean, and principal components if ZCA whitening is applied).
            datagen.fit(self.x_tr)

            steps_per_epoch =  math.ceil(len(self.x_tr) / batch_size)
            # fit the model on the batches generated by datagen.flow().
            self.model.fit(x=datagen.flow(self.x_tr, self.y_tr, batch_size=batch_size),
                      verbose=0,
                      epochs=epochs,
                      validation_data=(self.x_v, self.y_v),
                      steps_per_epoch=steps_per_epoch,
                      callbacks=callbacks)
        return  

In [47]:
from keras.datasets import cifar10
import keras

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# # building the input vector from the 32x32 pixels
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalizing the data to help with the training
x_train /= 255
x_test /= 255

# one-hot encoding using keras' numpy-related utilities
n_classes = 10
input_shape = (32,32,3)
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [48]:
y_train.shape

(40000, 10)

In [43]:
vgg = trainer('models/CIFAR10_vgg11.keras',x_train,y_train,x_val,y_val,'vgg11',10)
vgg.model.summary()
vgg.train(save_best_only=True,save_weights_only = False)
model = load_model('models/CIFAR10_vgg11.keras')
np.sum(np.argmax(model.predict(x_test),axis=-1) == np.argmax(y_test),axis=-1)/10000
vgg = trainer('models/CIFAR10_vgg13.keras',x_train,y_train,x_val,y_val,'vgg13',10)
vgg.model.summary()
vgg.train(save_best_only=True,save_weights_only = False)
model = load_model('models/CIFAR10_vgg13.keras')
np.sum(np.argmax(model.predict(x_test),axis=-1) == np.argmax(y_test),axis=-1)/10000
vgg = trainer('models/CIFAR10_vgg16.keras',x_train,y_train,x_val,y_val,'vgg16',10)
vgg.model.summary()
vgg.train(save_best_only=True,save_weights_only = False)
model = load_model('models/CIFAR10_vgg16.keras')
np.sum(np.argmax(model.predict(x_test),axis=-1) == np.argmax(y_test),axis=-1)/10000
vgg = trainer('models/CIFAR10_vgg19.keras',x_train,y_train,x_val,y_val,'vgg19',10)
vgg.model.summary()
vgg.train(save_best_only=True,save_weights_only = False)
model = load_model('models/CIFAR10_vgg19.keras')
np.sum(np.argmax(model.predict(x_test),axis=-1) == np.argmax(y_test),axis=-1)/10000

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_190 (Conv2D)         (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization_190 (Ba  (None, 32, 32, 64)       256       
 tchNormalization)                                               
                                                                 
 re_lu_190 (ReLU)            (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d_75 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_191 (Conv2D)         (None, 16, 16, 128)       7385


Epoch 30: val_acc did not improve from 0.87780

Epoch 31: val_acc did not improve from 0.87780

Epoch 32: val_acc did not improve from 0.87780

Epoch 33: val_acc did not improve from 0.87780

Epoch 34: val_acc did not improve from 0.87780

Epoch 35: val_acc did not improve from 0.87780

Epoch 36: val_acc improved from 0.87780 to 0.87810, saving model to models/CIFAR10_vgg11.keras

Epoch 37: val_acc did not improve from 0.87810

Epoch 38: val_acc did not improve from 0.87810

Epoch 39: val_acc improved from 0.87810 to 0.88310, saving model to models/CIFAR10_vgg11.keras

Epoch 40: val_acc did not improve from 0.88310

Epoch 41: val_acc did not improve from 0.88310

Epoch 42: val_acc did not improve from 0.88310

Epoch 43: val_acc did not improve from 0.88310

Epoch 44: val_acc did not improve from 0.88310

Epoch 45: val_acc did not improve from 0.88310

Epoch 46: val_acc did not improve from 0.88310

Epoch 47: val_acc did not improve from 0.88310

Epoch 48: val_acc did not improve from 


Epoch 177: val_acc did not improve from 0.90540

Epoch 178: val_acc did not improve from 0.90540

Epoch 179: val_acc did not improve from 0.90540

Epoch 180: val_acc did not improve from 0.90540

Epoch 181: val_acc did not improve from 0.90540

Epoch 182: val_acc did not improve from 0.90540

Epoch 183: val_acc did not improve from 0.90540

Epoch 184: val_acc did not improve from 0.90540

Epoch 185: val_acc did not improve from 0.90540

Epoch 186: val_acc improved from 0.90540 to 0.90580, saving model to models/CIFAR10_vgg11.keras

Epoch 187: val_acc did not improve from 0.90580

Epoch 188: val_acc did not improve from 0.90580

Epoch 189: val_acc did not improve from 0.90580

Epoch 190: val_acc did not improve from 0.90580

Epoch 191: val_acc did not improve from 0.90580

Epoch 192: val_acc did not improve from 0.90580

Epoch 193: val_acc did not improve from 0.90580

Epoch 194: val_acc did not improve from 0.90580

Epoch 195: val_acc did not improve from 0.90580

Epoch 196: val_acc d


Epoch 2: val_acc improved from 0.60430 to 0.71380, saving model to models/CIFAR10_vgg13.keras

Epoch 3: val_acc did not improve from 0.71380

Epoch 4: val_acc improved from 0.71380 to 0.76000, saving model to models/CIFAR10_vgg13.keras

Epoch 5: val_acc did not improve from 0.76000

Epoch 6: val_acc improved from 0.76000 to 0.78550, saving model to models/CIFAR10_vgg13.keras

Epoch 7: val_acc improved from 0.78550 to 0.80860, saving model to models/CIFAR10_vgg13.keras

Epoch 8: val_acc improved from 0.80860 to 0.83080, saving model to models/CIFAR10_vgg13.keras

Epoch 9: val_acc improved from 0.83080 to 0.84840, saving model to models/CIFAR10_vgg13.keras

Epoch 10: val_acc did not improve from 0.84840

Epoch 11: val_acc did not improve from 0.84840

Epoch 12: val_acc did not improve from 0.84840

Epoch 13: val_acc did not improve from 0.84840

Epoch 14: val_acc did not improve from 0.84840

Epoch 15: val_acc improved from 0.84840 to 0.87010, saving model to models/CIFAR10_vgg13.keras



Epoch 145: val_acc did not improve from 0.92190

Epoch 146: val_acc did not improve from 0.92190

Epoch 147: val_acc did not improve from 0.92190

Epoch 148: val_acc did not improve from 0.92190

Epoch 149: val_acc did not improve from 0.92190

Epoch 150: val_acc did not improve from 0.92190

Epoch 151: val_acc did not improve from 0.92190

Epoch 152: val_acc did not improve from 0.92190

Epoch 153: val_acc did not improve from 0.92190

Epoch 154: val_acc improved from 0.92190 to 0.92220, saving model to models/CIFAR10_vgg13.keras

Epoch 155: val_acc did not improve from 0.92220

Epoch 156: val_acc did not improve from 0.92220

Epoch 157: val_acc did not improve from 0.92220

Epoch 158: val_acc did not improve from 0.92220

Epoch 159: val_acc did not improve from 0.92220

Epoch 160: val_acc did not improve from 0.92220

Epoch 161: val_acc did not improve from 0.92220

Epoch 162: val_acc did not improve from 0.92220

Epoch 163: val_acc did not improve from 0.92220

Epoch 164: val_acc d

...


Epoch 88: val_acc did not improve from 0.91660

Epoch 89: val_acc improved from 0.91660 to 0.91810, saving model to models/CIFAR10_vgg16.keras

Epoch 90: val_acc did not improve from 0.91810

Epoch 91: val_acc did not improve from 0.91810

Epoch 92: val_acc did not improve from 0.91810

Epoch 93: val_acc did not improve from 0.91810

Epoch 94: val_acc did not improve from 0.91810

Epoch 95: val_acc improved from 0.91810 to 0.91930, saving model to models/CIFAR10_vgg16.keras

Epoch 96: val_acc did not improve from 0.91930

Epoch 97: val_acc did not improve from 0.91930

Epoch 98: val_acc improved from 0.91930 to 0.91990, saving model to models/CIFAR10_vgg16.keras

Epoch 99: val_acc did not improve from 0.91990

Epoch 100: val_acc did not improve from 0.91990

Epoch 101: val_acc did not improve from 0.91990

Epoch 102: val_acc did not improve from 0.91990

Epoch 103: val_acc did not improve from 0.91990

Epoch 104: val_acc did not improve from 0.91990

Epoch 105: val_acc did not improve

                                                                 
 max_pooling2d_106 (MaxPooli  (None, 8, 8, 128)        0         
 ng2D)                                                           
                                                                 
 conv2d_264 (Conv2D)         (None, 8, 8, 256)         295168    
                                                                 
 batch_normalization_264 (Ba  (None, 8, 8, 256)        1024      
 tchNormalization)                                               
                                                                 
 re_lu_264 (ReLU)            (None, 8, 8, 256)         0         
                                                                 
 conv2d_265 (Conv2D)         (None, 8, 8, 256)         590080    
                                                                 
 batch_normalization_265 (Ba  (None, 8, 8, 256)        1024      
 tchNormalization)                                               
          


Epoch 14: val_acc did not improve from 0.82170

Epoch 15: val_acc improved from 0.82170 to 0.84520, saving model to models/CIFAR10_vgg19.keras

Epoch 16: val_acc improved from 0.84520 to 0.85560, saving model to models/CIFAR10_vgg19.keras

Epoch 17: val_acc did not improve from 0.85560

Epoch 18: val_acc did not improve from 0.85560

Epoch 19: val_acc improved from 0.85560 to 0.85760, saving model to models/CIFAR10_vgg19.keras

Epoch 20: val_acc improved from 0.85760 to 0.87100, saving model to models/CIFAR10_vgg19.keras

Epoch 21: val_acc did not improve from 0.87100

Epoch 22: val_acc did not improve from 0.87100

Epoch 23: val_acc improved from 0.87100 to 0.87510, saving model to models/CIFAR10_vgg19.keras

Epoch 24: val_acc improved from 0.87510 to 0.87820, saving model to models/CIFAR10_vgg19.keras

Epoch 25: val_acc did not improve from 0.87820

Epoch 26: val_acc improved from 0.87820 to 0.87940, saving model to models/CIFAR10_vgg19.keras

Epoch 27: val_acc did not improve from 


Epoch 159: val_acc did not improve from 0.91640

Epoch 160: val_acc improved from 0.91640 to 0.91700, saving model to models/CIFAR10_vgg19.keras

Epoch 161: val_acc did not improve from 0.91700

Epoch 162: val_acc did not improve from 0.91700

Epoch 163: val_acc did not improve from 0.91700

Epoch 164: val_acc did not improve from 0.91700

Epoch 165: val_acc did not improve from 0.91700

Epoch 166: val_acc did not improve from 0.91700

Epoch 167: val_acc did not improve from 0.91700

Epoch 168: val_acc did not improve from 0.91700

Epoch 169: val_acc did not improve from 0.91700

Epoch 170: val_acc did not improve from 0.91700

Epoch 171: val_acc did not improve from 0.91700

Epoch 172: val_acc did not improve from 0.91700

Epoch 173: val_acc did not improve from 0.91700

Epoch 174: val_acc did not improve from 0.91700

Epoch 175: val_acc did not improve from 0.91700

Epoch 176: val_acc did not improve from 0.91700

Epoch 177: val_acc did not improve from 0.91700

Epoch 178: val_acc d

0.1323

In [57]:
model = load_model('models/CIFAR10_vgg11.keras')
np.sum(np.argmax(model.predict(x_test),axis=-1) == np.argmax(y_test,axis=-1))/10000


313/313 [==============================] - 0s 966us/step


0.7871

In [58]:
model = load_model('models/CIFAR10_vgg13.keras')
np.sum(np.argmax(model.predict(x_test),axis=-1) == np.argmax(y_test,axis=-1))/10000

313/313 [==============================] - 0s 1ms/step


0.8661

In [59]:
model = load_model('models/CIFAR10_vgg16.keras')
np.sum(np.argmax(model.predict(x_test),axis=-1) == np.argmax(y_test,axis=-1))/10000

313/313 [==============================] - 1s 1ms/step


0.8599

In [60]:
model = load_model('models/CIFAR10_vgg19.keras')
np.sum(np.argmax(model.predict(x_test),axis=-1) == np.argmax(y_test,axis=-1))/10000

313/313 [==============================] - 1s 2ms/step


0.8577

In [54]:
import sys, os

In [55]:
sys.path.append("/projectnb/nonarch/adversarial_experiment/")


In [56]:
import cleverhans

from keras.datasets import cifar10
import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input
from tensorflow.keras.layers import Flatten, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
import numpy as np
import math
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model


In [57]:
adv_experiment_path = '/projectnb/nonarch/adversarial_experiment/'
adv_example_path = adv_experiment_path + 'adversarial_examples/gen_by_vgg16'

In [28]:
GENmodel = load_model('models/CIFAR10_vgg16.keras') 
cifar10_dataset = cifar10.load_data()
training, test = cifar10_dataset
x_tr, x_v, y_tr, y_v = train_test_split(training[0], training[1], test_size=0.2, random_state=42)
x_tr = x_tr.reshape(x_tr.shape[0],32,32,3)
x_v = x_v.reshape(x_v.shape[0],32,32,3)
x_v = x_v.astype('float32')
x_tr = x_tr.astype('float32')
x_v /= 255
x_tr /= 255
y_tr = keras.utils.to_categorical(y_tr, 10)
y_v = keras.utils.to_categorical(y_v,10)
x_test, y_test = test
x_test = x_test.reshape(x_test.shape[0],32,32,3)
x_test = x_test.astype('float32')
x_test /= 255 
y_test = keras.utils.to_categorical(y_test, 10)

from cleverhans.tf2.attacks import projected_gradient_descent as pgd

eps = 216/255 # value ranges between 0 - 255. 
nb_iter = 8
norm = 2
targeted = False
eps_iter = 96/255
advs_x =[]
advs_y =[]
label = np.round(eps_iter,3)
for i in range(16):
    print(f'{2500*i} - {2500*(i+1)}', end = ' ')
    adv = pgd.projected_gradient_descent(GENmodel, x_tr[2500*i:2500*(i+1)],eps,eps_iter,nb_iter,norm,y=tf.argmax(y_tr[2500*i:2500*(i+1)],1))
    advs_x.append(adv) 
advs_x = np.concatenate(advs_x, axis=0)
print(f"advs_x shape: {advs_x.shape}")
np.save(f'{adv_example_path}/pgd_{label}_x_untarget.npy', advs_x)
print(f'saved to adversarial_examples with file name "pgd_{label}_x_untarget.npy" ')
    
logits = GENmodel.predict(x_tr, batch_size=2000)
least_likely = tf.argmin(logits,axis=-1)

# Targetted

eps = 216/255 # value ranges between 0 - 255. allow to perturbe 216 at most.
# eps_iters = [3/255,9/255,27/255,48/255,96/255]
eps_iter = 96/255
nb_iter = 8
norm = 2
targeted = True

y= least_likely
eps_iter = 96/255
advs_x =[]
advs_y =[]
label = np.round(eps_iter,3)
print(f'epsilon = {eps_iter}')
for i in range(16):
    print(f'{2500*i} - {2500*(i+1)}', end = ' ')
    adv = pgd.projected_gradient_descent(GENmodel,x_tr[2500*i:2500*(i+1)],eps,eps_iter,nb_iter,norm,y=y[2500*i:2500*(i+1)],targeted=targeted)
    advs_x.append(adv) 
advs_x = np.concatenate(advs_x, axis=0)
print(f"advs_x shape: {advs_x.shape}")
np.save(f'{adv_example_path}/pgd_{label}_x_target_to_ll.npy', advs_x)
print(f'saved to adversarial_examples with file name "pgd_{label}_x_target_to_ll.npy" ')

20/20 [==============================] - 1s 38ms/step
epsilon = 0.3764705882352941
0 - 2500 2500 - 5000 5000 - 7500 7500 - 10000 10000 - 12500 12500 - 15000 15000 - 17500 17500 - 20000 20000 - 22500 22500 - 25000 25000 - 27500 27500 - 30000 30000 - 32500 32500 - 35000 35000 - 37500 37500 - 40000 advs_x shape: (40000, 32, 32, 3)
saved to adversarial_examples with file name "pgd_0.376_x_target_to_ll.npy" 


In [51]:
x_tr, x_v, y_tr, y_v = train_test_split(training[0], training[1], test_size=0.2, random_state=42)
x_tr = x_tr.reshape(x_tr.shape[0],32,32,3)
x_v = x_v.reshape(x_v.shape[0],32,32,3)
x_v = x_v.astype('float32')
x_tr = x_tr.astype('float32')
x_v /= 255
x_tr /= 255
# y_tr = keras.utils.to_categorical(y_tr, 10)
# y_v = keras.utils.to_categorical(y_v,10)
x_test, y_test = test
x_test = x_test.reshape(x_test.shape[0],32,32,3)
x_test = x_test.astype('float32')
x_test /= 255 
y_test = keras.utils.to_categorical(y_test, 10)

adv_x_tr = np.load(f"{adv_experiment_path}adversarial_examples/pgd_0.376_x_untarget.npy")
adv_x_val = np.load(f"{adv_experiment_path}adversarial_examples/pgd_0.376_x_val_untarget.npy")
# print(f'{adversarial_direction} adversarial dataset has been loaded.')
path = f'adversarial_models/untargeted_trained_20targets_vgg16.keras'
training_x = np.concatenate([x_tr,adv_x_tr],axis=0)
validating_x = np.concatenate([x_v,adv_x_val],axis=0)
training_y_long=np.concatenate([y_tr,y_tr+10],axis=0)
training_y_long = keras.utils.to_categorical(training_y_long,20)
validating_y_long=np.concatenate([y_v,y_v+10],axis=0)
validating_y_long = keras.utils.to_categorical(validating_y_long,20)


(80000, 20)

In [53]:
vgg = trainer(path,training_x,training_y_long,validating_x,validating_y_long,'vgg16',20)
vgg.model.summary()
vgg.train(save_best_only=True,save_weights_only = False)
model = load_model(path)



Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_26 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization_26 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                                 
 re_lu_26 (ReLU)             (None, 32, 32, 64)        0         
                                                                 
 conv2d_27 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 batch_normalization_27 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                          


Epoch 1: val_acc improved from -inf to 0.30205, saving model to adversarial_models/untargeted_trained_20targets_vgg16.keras

Epoch 2: val_acc improved from 0.30205 to 0.33130, saving model to adversarial_models/untargeted_trained_20targets_vgg16.keras

Epoch 3: val_acc improved from 0.33130 to 0.38780, saving model to adversarial_models/untargeted_trained_20targets_vgg16.keras

Epoch 4: val_acc improved from 0.38780 to 0.38945, saving model to adversarial_models/untargeted_trained_20targets_vgg16.keras

Epoch 5: val_acc did not improve from 0.38945

Epoch 6: val_acc improved from 0.38945 to 0.42815, saving model to adversarial_models/untargeted_trained_20targets_vgg16.keras

Epoch 7: val_acc improved from 0.42815 to 0.43665, saving model to adversarial_models/untargeted_trained_20targets_vgg16.keras

Epoch 8: val_acc improved from 0.43665 to 0.45095, saving model to adversarial_models/untargeted_trained_20targets_vgg16.keras

Epoch 9: val_acc improved from 0.45095 to 0.77980, saving m


Epoch 124: val_acc did not improve from 0.91280

Epoch 125: val_acc did not improve from 0.91280

Epoch 126: val_acc did not improve from 0.91280

Epoch 127: val_acc did not improve from 0.91280

Epoch 128: val_acc did not improve from 0.91280

Epoch 129: val_acc did not improve from 0.91280

Epoch 130: val_acc did not improve from 0.91280

Epoch 131: val_acc did not improve from 0.91280

Epoch 132: val_acc did not improve from 0.91280

Epoch 133: val_acc did not improve from 0.91280

Epoch 134: val_acc did not improve from 0.91280

Epoch 135: val_acc did not improve from 0.91280

Epoch 136: val_acc improved from 0.91280 to 0.91310, saving model to adversarial_models/untargeted_trained_20targets_vgg16.keras

Epoch 137: val_acc did not improve from 0.91310

Epoch 138: val_acc did not improve from 0.91310

Epoch 139: val_acc did not improve from 0.91310

Epoch 140: val_acc did not improve from 0.91310

Epoch 141: val_acc did not improve from 0.91310

Epoch 142: val_acc did not improve f

In [58]:
GENmodel = load_model('models/CIFAR10_vgg16.keras') 
cifar10_dataset = cifar10.load_data()
training, test = cifar10_dataset
x_tr, x_v, y_tr, y_v = train_test_split(training[0], training[1], test_size=0.2, random_state=42)
x_tr = x_tr.reshape(x_tr.shape[0],32,32,3)
x_v = x_v.reshape(x_v.shape[0],32,32,3)
x_v = x_v.astype('float32')
x_tr = x_tr.astype('float32')
x_v /= 255
x_tr /= 255
y_tr = keras.utils.to_categorical(y_tr, 10)
y_v = keras.utils.to_categorical(y_v,10)
x_test, y_test = test
x_test = x_test.reshape(x_test.shape[0],32,32,3)
x_test = x_test.astype('float32')
x_test /= 255 
y_test = keras.utils.to_categorical(y_test, 10)

from cleverhans.tf2.attacks import projected_gradient_descent as pgd

eps = 216/255 # value ranges between 0 - 255. 
nb_iter = 8
norm = 2
targeted = False
eps_iter = 96/255
advs_x =[]
advs_y =[]
label = np.round(eps_iter,3)
for i in range(4):
    print(f'{2500*i} - {2500*(i+1)}', end = ' ')
    adv = pgd.projected_gradient_descent(GENmodel, x_v[2500*i:2500*(i+1)],eps,eps_iter,nb_iter,norm,y=tf.argmax(y_tr[2500*i:2500*(i+1)],1))
    advs_x.append(adv) 
advs_x = np.concatenate(advs_x, axis=0)
print(f"advs_x_v shape: {advs_x.shape}")
np.save(f'{adv_example_path}/pgd_{label}_x_val_untarget.npy', advs_x)
print(f'saved to adversarial_examples with file name "pgd_{label}_x_val_untarget.npy" ')
    
logits = GENmodel.predict(x_v, batch_size=2000)
least_likely = tf.argmin(logits,axis=-1)

# Targetted

eps = 216/255 # value ranges between 0 - 255. allow to perturbe 216 at most.
# eps_iters = [3/255,9/255,27/255,48/255,96/255]
eps_iter = 96/255
nb_iter = 8
norm = 2
targeted = True

y= least_likely
eps_iter = 96/255
advs_x =[]
advs_y =[]
label = np.round(eps_iter,3)
print(f'epsilon = {eps_iter}')
for i in range(4):
    print(f'{2500*i} - {2500*(i+1)}', end = ' ')
    adv = pgd.projected_gradient_descent(GENmodel,x_v[2500*i:2500*(i+1)],eps,eps_iter,nb_iter,norm,y=y[2500*i:2500*(i+1)],targeted=targeted)
    advs_x.append(adv) 
advs_x = np.concatenate(advs_x, axis=0)
print(f"advs_x_v shape: {advs_x.shape}")
np.save(f'{adv_example_path}/pgd_{label}_x_val_target_to_ll.npy', advs_x)
print(f'saved to adversarial_examples with file name "pgd_{label}_x_val_target_to_ll.npy" ')


eps = 216/255 # value ranges between 0 - 255. 
nb_iter = 8
norm = 2
targeted = False
eps_iter = 96/255
advs_x =[]
advs_y =[]
label = np.round(eps_iter,3)
for i in range(4):
    print(f'{2500*i} - {2500*(i+1)}', end = ' ')
    adv = pgd.projected_gradient_descent(GENmodel, x_test[2500*i:2500*(i+1)],eps,eps_iter,nb_iter,norm,y=tf.argmax(y_tr[2500*i:2500*(i+1)],1))
    advs_x.append(adv) 
advs_x = np.concatenate(advs_x, axis=0)
print(f"advs_x_test shape: {advs_x.shape}")
np.save(f'{adv_example_path}/pgd_{label}_x_test_untarget.npy', advs_x)
print(f'saved to adversarial_examples with file name "pgd_{label}_x_test_untarget.npy" ')
    
logits = GENmodel.predict(x_test, batch_size=2000)
least_likely = tf.argmin(logits,axis=-1)

# Targetted

eps = 216/255 # value ranges between 0 - 255. allow to perturbe 216 at most.
# eps_iters = [3/255,9/255,27/255,48/255,96/255]
eps_iter = 96/255
nb_iter = 8
norm = 2
targeted = True

y= least_likely
eps_iter = 96/255
advs_x =[]
advs_y =[]
label = np.round(eps_iter,3)
print(f'epsilon = {eps_iter}')
for i in range(4):
    print(f'{2500*i} - {2500*(i+1)}', end = ' ')
    adv = pgd.projected_gradient_descent(GENmodel,x_test[2500*i:2500*(i+1)],eps,eps_iter,nb_iter,norm,y=y[2500*i:2500*(i+1)],targeted=targeted)
    advs_x.append(adv) 
advs_x = np.concatenate(advs_x, axis=0)
print(f"advs_x_test shape: {advs_x.shape}")
np.save(f'{adv_example_path}/pgd_{label}_x_test_target_to_ll.npy', advs_x)
print(f'saved to adversarial_examples with file name "pgd_{label}_x_test_target_to_ll.npy" ')

0 - 2500 2500 - 5000 5000 - 7500 7500 - 10000 advs_x_v shape: (10000, 32, 32, 3)
saved to adversarial_examples with file name "pgd_0.376_x_val_untarget.npy" 
5/5 [==============================] - 0s 39ms/step
epsilon = 0.3764705882352941
0 - 2500 2500 - 5000 5000 - 7500 7500 - 10000 advs_x_v shape: (10000, 32, 32, 3)
saved to adversarial_examples with file name "pgd_0.376_x_val_target_to_ll.npy" 
0 - 2500 2500 - 5000 5000 - 7500 7500 - 10000 advs_x_test shape: (10000, 32, 32, 3)
saved to adversarial_examples with file name "pgd_0.376_x_test_untarget.npy" 
5/5 [==============================] - 0s 39ms/step
epsilon = 0.3764705882352941
0 - 2500 2500 - 5000 5000 - 7500 7500 - 10000 advs_x_test shape: (10000, 32, 32, 3)
saved to adversarial_examples with file name "pgd_0.376_x_test_target_to_ll.npy" 


In [59]:
# adv_experiment_path = '/projectnb/nonarch/adversarial_experiment/'
# adv_example_path = adv_experiment_path + 'adversarial_examples/gen_by_vgg16'

x_tr, x_v, y_tr, y_v = train_test_split(training[0], training[1], test_size=0.2, random_state=42)
x_tr = x_tr.reshape(x_tr.shape[0],32,32,3)
x_v = x_v.reshape(x_v.shape[0],32,32,3)
x_v = x_v.astype('float32')
x_tr = x_tr.astype('float32')
x_v /= 255
x_tr /= 255
# y_tr = keras.utils.to_categorical(y_tr, 10)
# y_v = keras.utils.to_categorical(y_v,10)
x_test, y_test = test
x_test = x_test.reshape(x_test.shape[0],32,32,3)
x_test = x_test.astype('float32')
x_test /= 255 
y_test = keras.utils.to_categorical(y_test, 10)

adv_x_tr = np.load(f"{adv_example_path}/pgd_0.376_x_untarget.npy")
adv_x_val = np.load(f"{adv_example_path}/pgd_0.376_x_val_untarget.npy")
# print(f'{adversarial_direction} adversarial dataset has been loaded.')
path = f'adversarial_models/untargeted_trained_20targets_vgg16.keras'
training_x = np.concatenate([x_tr,adv_x_tr],axis=0)
validating_x = np.concatenate([x_v,adv_x_val],axis=0)
training_y_long=np.concatenate([y_tr,y_tr+10],axis=0)
training_y_long = keras.utils.to_categorical(training_y_long,20)
validating_y_long=np.concatenate([y_v,y_v+10],axis=0)
validating_y_long = keras.utils.to_categorical(validating_y_long,20)


In [60]:

path = f'{adv_experiment_path}adversarial_models/vgg/untargeted_trained_20targets_vgg16.keras'

In [61]:
vgg = trainer(path,training_x,training_y_long,validating_x,validating_y_long,'vgg16',20)
vgg.model.summary()
vgg.train(save_best_only=True,save_weights_only = False)
model = load_model(path)



Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_39 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization_39 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                                 
 re_lu_39 (ReLU)             (None, 32, 32, 64)        0         
                                                                 
 conv2d_40 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 batch_normalization_40 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                          


Epoch 1: val_acc improved from -inf to 0.29675, saving model to /projectnb/nonarch/adversarial_experiment/adversarial_models/vgg/untargeted_trained_20targets_vgg16.keras

Epoch 2: val_acc improved from 0.29675 to 0.31305, saving model to /projectnb/nonarch/adversarial_experiment/adversarial_models/vgg/untargeted_trained_20targets_vgg16.keras

Epoch 3: val_acc improved from 0.31305 to 0.35510, saving model to /projectnb/nonarch/adversarial_experiment/adversarial_models/vgg/untargeted_trained_20targets_vgg16.keras

Epoch 4: val_acc improved from 0.35510 to 0.37585, saving model to /projectnb/nonarch/adversarial_experiment/adversarial_models/vgg/untargeted_trained_20targets_vgg16.keras

Epoch 5: val_acc improved from 0.37585 to 0.38920, saving model to /projectnb/nonarch/adversarial_experiment/adversarial_models/vgg/untargeted_trained_20targets_vgg16.keras

Epoch 6: val_acc improved from 0.38920 to 0.40290, saving model to /projectnb/nonarch/adversarial_experiment/adversarial_models/vgg/


Epoch 95: val_acc did not improve from 0.80120

Epoch 96: val_acc did not improve from 0.80120

Epoch 97: val_acc improved from 0.80120 to 0.80395, saving model to /projectnb/nonarch/adversarial_experiment/adversarial_models/vgg/untargeted_trained_20targets_vgg16.keras

Epoch 98: val_acc did not improve from 0.80395

Epoch 99: val_acc did not improve from 0.80395

Epoch 100: val_acc did not improve from 0.80395

Epoch 101: val_acc did not improve from 0.80395

Epoch 102: val_acc did not improve from 0.80395

Epoch 103: val_acc did not improve from 0.80395

Epoch 104: val_acc did not improve from 0.80395

Epoch 105: val_acc did not improve from 0.80395

Epoch 106: val_acc improved from 0.80395 to 0.80745, saving model to /projectnb/nonarch/adversarial_experiment/adversarial_models/vgg/untargeted_trained_20targets_vgg16.keras

Epoch 107: val_acc did not improve from 0.80745

Epoch 108: val_acc did not improve from 0.80745

Epoch 109: val_acc did not improve from 0.80745

Epoch 110: val_